## Read Data and Packages

In [2]:
import pandas as pd

In [ ]:
from pydantic import BaseModel, Field
from typing import Optional, Literal, List, Dict

#create different class for each level 
class Metrics(BaseModel):
    name: str
    value: str

class Indicators(BaseModel):
    name: str
    assessment: str
    metrics: Optional[list[Metrics]] = None
    source: Optional[str] = None

class Area(BaseModel):
    name: str
    assessment: Optional[str] = None
    indicators: List[Indicators]

class Pillars(BaseModel):
    name: str
    areas: List[Area]

class Metadata(BaseModel):
    country: str
    assessment_year: int

In [3]:
df_assessments = pd.read_excel("C:/Users/User/OneDrive/Documents/GitHub/DS205/Week 2/Lab 2/ascor-api/data/TPI ASCOR data - 13012025/ASCOR_assessments_results.xlsx")
df_assessments['Assessment date'] = pd.to_datetime(df_assessments['Assessment date'])
df_assessments['Publication date'] = pd.to_datetime(df_assessments['Publication date'])

C:\Users\User\AppData\Local\Temp\ipykernel_13128\1045124698.py:2: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_assessments['Assessment date'] = pd.to_datetime(df_assessments['Assessment date'])


## Country Endpoints Function Prototype

In [22]:
def get_country_data(country: str, assessment_year: int):

    # Filter the data by request
    selected_row = (
        (df_assessments["Country"] == country) &
        (df_assessments['Assessment date'].dt.year == assessment_year)
    )

    data = df_assessments[selected_row]

     #if data.empty:
        #raise HTTPException(status_code=404, 
                           # detail=f"There is no data for country: {country} and year: {assessment_year}")
    
    #replace NAs as not valid in Json
    data = data.fillna('')

    #remove . in column names
    remap_column_names = {
        col: col.replace('.', '_')
        for col in data.columns
    }
    data.rename(columns=remap_column_names, inplace=True)

   #extract pillars
    
    def get_pillars(data):     #pillar is either CP or EP or CF
        
    def get_area(pillar, data):
        pillar_name = pillar.name()
        area_columns = [col for col in data.columns if pillar_name and 'area' in col]
        
        for area in area_columns:
            area_name = area[-4:]
            assesment = data[area]
            indicator = get_indicator(area, data)
            individual_area = Area(name = area_name, assessment = assesment, indicators = indicator)


   
   
    #extract indicators

    indicator_columns = [col for col in data.columns if col.startswith('indicator')]

    
    def get_indicator(area, data):
        area_name = #make sure to extract string of name
        area_columns = [col for col in data.columns if area in col]

        for area in area_columns:
            area = area[-4:]
            indicators = []
            for col in data.columns:
                if area and "indicator" in col:
                    indicators.append(col)
        
        for indicator in data[indicators]:
            indicator_list = []
            indicator_name = indicator[-6:]
            assesment = 

            individual_indicator = Indicator(name=indicator_name, assesment = assesment, metrics = )
            indicator_lsit.append(individual_indicator)
        return indicator_lsit
   
   
    def get_metrics(indicator, data):
        
        for indicator in indicator_columns:
            indicator = indicator[-6:]
        
            individual_indicator = Indicator
        
            metric_columns = [col for col in data.columns if indicator in col]
        metric_columns = [col for col in data.columns if indicator in col]  

        for name, value in data[metric_columns].to_dict().items():
            metric_list = []
            individual_metric = Metric(name=name, value=value)
            metric_list.append(individual_metric)
        return metric_list


    EP = {col: data[col] for col in data.index if col.startswith("EP")}
    CP = {col: data[col] for col in data.index if col.startswith("CP")}
    CF = {col: data[col] for col in data.index if col.startswith("CF")}

    # Selected and filter columns
    #area_columns = [col for col in df_assessments.columns if col.startswith("area")]
    #data = data[area_columns]


#make output dictionary mirror model
    output_dict = {
       "country" : country,
       "assessment_year" : assessment_year,
        
        "pillars" : {
           "name": pillar_name,
            
            "areas" : {
               "name" : area_name,
                "assessment" : area_assessment, 
                
                "indicators" : {
                   "name" : indicator_name,
                   "assessment" : indicator_assessment,
                  
                   "metrics" : {
                      "name" : metric_name,
                      "value" : metric_value
                   }
                }
            }
        }   
    }

    output = CountryData(**output_dict)

    return output

In [12]:
country = "Brazil"
assessment_year = 2023

selected_row = (
      (df_assessments["Country"] == country) &
      (df_assessments['Assessment date'].dt.year == assessment_year)
    )

    # Filter the data
data = df_assessments[selected_row]
#data = data.iloc[0]   

data.head(30)

,Id,Assessment date,Publication date,Country Id,Country,area EP.1,indicator EP.1.a,indicator EP.1.b,indicator EP.1.c,area EP.2,...,year metric CP.4.d.i,year metric CP.4.e.i,year metric CP.6.a.i,year metric CF.1.a.i,year metric CF.1.b.i,year metric CF.4.i,year metric CF.4.ii,year metric CF.4.iii,year metric CF.4.iv,Notes
3,91,2023-10-31,2023-01-12,18,Brazil,Partial,Yes,No,No,Partial,...,2020.0,2022.0,2021,NaN,NaN,2023,2023,2023,2023,Assessment results are based on in-depth polic...
